<a href="https://colab.research.google.com/github/itinstructor/JupyterNotebooks/blob/main/Notebooks%5CFlaskServerIPGeolocation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Of course. You can display a map of user locations by getting their IP address, converting it to geographic coordinates using a geolocation service, storing those coordinates, and then plotting them on a map using a front-end library.

Since you're running Flask behind an IIS reverse proxy, the key is to get the user's *actual* IP from the `X-Forwarded-For` header, not the IP of the IIS server itself.

Here’s a complete guide to achieve this.

-----

## 1\. Get the User's IP Address

When a request goes through a proxy like IIS, the original user's IP is stored in a header, typically `X-Forwarded-For`. In Flask, you can access this from the `request` object.

A reliable way to get the IP is to check for the header first and fall back to `request.remote_addr` if it's not present.

In [ ]:
from flask import request

def get_ip():
    # Check for the X-Forwarded-For header, which IIS should add
    if request.headers.getlist("X-Forwarded-For"):
        # The header can be a comma-separated list, take the first IP
        ip = request.headers.getlist("X-Forwarded-For")[0].split(',')[0]
    else:
        # If the header is not present, fall back to the remote address
        ip = request.remote_addr
    return ip

-----

## 2\. Convert IP to Geolocation

Once you have the IP address, you need a service to look up its location. You have two main options: a web service API or a local database.

### Option A: Use a Web Service (e.g., ipinfo.io)

This is the easiest method. You make an API call to a service, and it returns location data in JSON format. The `ipinfo.io` service has a generous free tier.

First, install the `requests` library:
`pip install requests`

Then, create a function to get the location:

In [ ]:
import requests

def get_location(ip_address):
    try:
        # For local testing, you might get a private IP (e.g., 127.0.0.1)
        # The API will ignore it, so you can return default values.
        if ip_address == "127.0.0.1":
            return {"city": "Localhost", "region": "N/A", "country": "N/A", "loc": "0,0"}

        response = requests.get(f'https://ipinfo.io/{ip_address}/json')
        response.raise_for_status() # Raise an exception for bad status codes
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching geolocation: {e}")
        return None

### Option B: Use a Local Database (MaxMind GeoLite2)

This method is faster, works offline, and has no rate limits, but it requires you to download a database file.

1.  Download the free **GeoLite2 City** database from [MaxMind's website](https://dev.maxmind.com/geoip/geolite2-free-geolocation-data). You'll need to sign up for a free account.
2.  Unzip the file and place the `GeoLite2-City.mmdb` file in your project directory.
3.  Install the official Python library: `pip install geoip2`

Then, create a function to query the database:

In [ ]:
import geoip2.database

def get_location_local(ip_address):
    # This assumes the DB file is in the same directory
    try:
        with geoip2.database.Reader('GeoLite2-City.mmdb') as reader:
            # Again, handle private IPs for local testing
            if ip_address == "127.0.0.1":
                return {"lat": 0, "lon": 0, "city": "Localhost"}

            response = reader.city(ip_address)
            return {
                "lat": response.location.latitude,
                "lon": response.location.longitude,
                "city": response.city.name
            }
    except (geoip2.errors.AddressNotFoundError, FileNotFoundError) as e:
        print(f"Error with local GeoIP database: {e}")
        return None

-----

## 3\. Store and Retrieve Location Data

You'll need a simple database to store the coordinates of each visit. We'll use **Flask-SQLAlchemy** with SQLite for simplicity.

Install the library:
`pip install Flask-SQLAlchemy`

In your main Flask app file (`app.py`), set up the database and define a simple model to store the locations.

In [ ]:
from flask import Flask, render_template, jsonify
from flask_sqlalchemy import SQLAlchemy
import requests # Or import geoip2.database

# ... (include get_ip and get_location functions from above) ...

app = Flask(__name__)
# Configure the SQLite database
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///locations.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db = SQLAlchemy(app)

# Database model for storing locations
class Location(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    lat = db.Column(db.Float, nullable=False)
    lon = db.Column(db.Float, nullable=False)
    city = db.Column(db.String(100))

# Create the database tables
with app.app_context():
    db.create_all()

-----

## 4\. Putting It All Together

Now, let's create the Flask routes. One route (`/`) will capture the user's location, and another (`/map`) will display the map.

### Full `app.py` Example

In [ ]:
from flask import Flask, render_template, request, jsonify
from flask_sqlalchemy import SQLAlchemy
import requests

# --- App and Database Setup ---
app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///locations.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db = SQLAlchemy(app)

# --- Database Model ---
class Location(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    lat = db.Column(db.Float, nullable=False)
    lon = db.Column(db.Float, nullable=False)
    city = db.Column(db.String(100))

# --- Helper Functions ---
def get_ip():
    if request.headers.getlist("X-Forwarded-For"):
        ip = request.headers.getlist("X-Forwarded-For")[0].split(',')[0]
    else:
        ip = request.remote_addr
    return ip

def get_location(ip_address):
    if ip_address == "127.0.0.1": # Handle local testing
        return {"lat": 40.7128, "lon": -74.0060, "city": "New York (Default)"}
    try:
        response = requests.get(f'https://ipinfo.io/{ip_address}/json')
        response.raise_for_status()
        data = response.json()
        if 'loc' in data:
            lat, lon = data['loc'].split(',')
            return {"lat": float(lat), "lon": float(lon), "city": data.get('city', 'Unknown City')}
    except Exception as e:
        print(f"Could not get location for IP {ip_address}: {e}")
    return None

# --- Flask Routes ---
@app.route("/")
def index():
    # 1. Get user's IP
    user_ip = get_ip()

    # 2. Get location from IP
    location_data = get_location(user_ip)

    # 3. Save to database if found
    if location_data:
        new_location = Location(
            lat=location_data['lat'],
            lon=location_data['lon'],
            city=location_data['city']
        )
        db.session.add(new_location)
        db.session.commit()

    return "Hello! Your location has been noted. <a href='/map'>View the map</a>."

@app.route("/map")
def show_map():
    # Render the map page
    return render_template('map.html')

@app.route("/api/locations")
def get_locations_data():
    # This API endpoint will be called by our JavaScript on the map page
    locations = Location.query.all()
    locations_list = [
        {"lat": loc.lat, "lon": loc.lon, "city": loc.city} for loc in locations
    ]
    return jsonify(locations_list)

# --- Main Entry Point ---
if __name__ == '__main__':
    with app.app_context():
        db.create_all()
    app.run(debug=True)

### The Map Page (`templates/map.html`)

For the front end, we'll use **Leaflet.js**, a popular open-source mapping library. It's free and simple to use. 🗺️

Create a folder named `templates` in your project directory and add this `map.html` file inside it.

```html
<!DOCTYPE html>
<html>
<head>
    <title>User Connection Map</title>
    <meta charset="utf-g" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    
    <link rel="stylesheet" href="https://unpkg.com/leaflet@1.9.4/dist/leaflet.css"
     integrity="sha256-p4NxAoJBhIIN+hmNHrzRCf9tD/miZyoHS5obTRR9BMY="
     crossorigin=""/>
     
    <script src="https://unpkg.com/leaflet@1.9.4/dist/leaflet.js"
     integrity="sha256-20nQCchB9co0qIjJZRGuk2/Z9VM+kNiyxNV1lvTlZBo="
     crossorigin=""></script>
     
    <style>
        /* Make the map fill the entire page */
        html, body, #map {
            height: 100%;
            width: 100%;
            margin: 0;
            padding: 0;
        }
    </style>
</head>
<body>

<div id="map"></div>

<script>
    // 1. Initialize the map and set its view to a default location and zoom level
    const map = L.map('map').setView([20, 0], 2); // Centered roughly globally

    // 2. Add a tile layer to the map (the map background)
    // We're using OpenStreetMap, which is free to use.
    L.tileLayer('https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', {
        attribution: '&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors'
    }).addTo(map);

    // 3. Fetch location data from our Flask API endpoint
    fetch('/api/locations')
        .then(response => response.json())
        .then(locations => {
            // 4. Add a marker for each location
            locations.forEach(loc => {
                if (loc.lat !== null && loc.lon !== null) {
                    const marker = L.marker([loc.lat, loc.lon]).addTo(map);
                    // Add a popup to the marker
                    marker.bindPopup(`<b>City:</b> ${loc.city || 'Unknown'}`);
                }
            });
        })
        .catch(error => console.error('Error fetching location data:', error));
</script>

</body>
</html>
```

### Project Structure

Your final project should look like this:

```
/my-flask-app
|-- app.py
|-- locations.db      (will be created automatically)
|-- GeoLite2-City.mmdb (if you use the local DB option)
|-- /templates
|   |-- map.html
```

-----

## Final Steps & Considerations

  * **Run the App:** Run your Flask app through Waitress/IIS. When users visit the main page (`/`), their location will be logged. When you visit `/map`, you'll see all the logged locations plotted.
  * **Privacy ⚠️:** Be transparent with your users\! IP geolocation is personally identifiable information (PII) in many jurisdictions (like under GDPR). **Always inform users** that you are collecting their location data and for what purpose. Consider anonymizing the data by slightly randomizing coordinates or only storing city-level data.
  * **Accuracy:** IP geolocation is not always precise. It usually identifies the city or the location of the Internet Service Provider (ISP), not the user's exact street address.